# 千帆function_call工具调用

## 简介

上一节[千帆function_call入门](./function_call.ipynb)展示了实现chat调用函数的功能，本节将介绍如何让chat与千帆工具进行交互，并编写更便利的调用函数。

## 准备

本文使用的模块如下：
- 千帆 Python SDK中的 chat_completion 模块，该模块提供了与千帆对话引擎的交互接口；
- 千帆 tool中的duckduckgo_search_tool，该工具提供了网络搜索功能。

首先安装千帆 Python SDK，版本号 >= 0.2.6。

In [85]:
%%capture
!pip install "qianfan>=0.2.6" -U

初始化我们所需要的凭证

In [86]:
# 初始化LLM
import os

# qianfan sdk 鉴权
# os.environ["QIANFAN_ACCESS_KEY"]="..."
# os.environ["QIANFAN_SECRET_KEY"]="..."


自定义函数库，此处调用qianfan.common.tool中的duckduckgo搜索引擎工具进行搜索，编写today工具查询今天日期。

In [87]:
from datetime import date
from typing import List

from qianfan.common.tool.base_tool import BaseTool, ToolParameter
from qianfan.common.tool import duckduckgo_search_tool
import json


class TodayTool(BaseTool):
    name: str = "today"
    description: str = "获取今天的日期，格式为YYYY-MM-DD"
    parameters: List[ToolParameter] = []

    def run(self, parameters: dict = {}) -> dict:
        return {
            "date": date.today().strftime("%Y-%m-%d")
        }


将工具添加到函数库中并定义函数名映射。

In [88]:
search_tool = duckduckgo_search_tool.DuckDuckGoSearchTool()
today_tool = TodayTool()

funcs = [
    search_tool.to_function_call_schema(),
    TodayTool().to_function_call_schema()
]

func_name_map = {
    search_tool.name:search_tool,
    today_tool.name:today_tool
}

将对话过程封装成函数，方便多轮对话调用

In [89]:
import qianfan

def chat(chat_completion, messages, functions, function_map):
    resp = chat_completion.do(
        messages=messages,
        functions=functions
    )
    
    if resp.get("function_call"):
        func_call = resp["function_call"]
        func_name = func_call["name"]
        arguments = json.loads(func_call["arguments"])
        func_content_dict = function_map[func_name].run(arguments)
        func_content = json.dumps(func_content_dict,ensure_ascii=False)
        
        messages.append(resp, role="assistant")
        messages.append(func_content, role="function")
        
        second_resp = chat_completion.do(
            messages=messages,
            functions=functions
        )
        resp_content = second_resp["result"]
        
        print("**call function {}**".format(func_name))
        print("function arguments: {}".format(arguments))
        # print("function result: {}".format(func_content))
    else:
        print("**No function**")
        resp_content = resp["result"]
    print(resp_content)
    msgs.append(resp_content, role="assistant")

开始对话，首先获取今天的日期。

In [90]:
chat_comp = qianfan.ChatCompletion(model="ERNIE-Bot-4")
query = "今天是几号？"
msgs = qianfan.QfMessages()
msgs.append(query,role="user")
chat(chat_comp, msgs, funcs, func_name_map)

**call function today**
function arguments: {}
今天是2024年1月15日。如果您还有其他问题或需要更多信息，请随时告诉我。


接下来进行多轮对话，用日期信息来查询新闻。

In [91]:
query = "那么根据刚才的日期昨天有什么新闻么？搜索并请列举出标题、摘要"
msgs.append(query,role="user")
chat(chat_comp, msgs, funcs, func_name_map)

**call function duckduckgo_search**
function arguments: {'search_query': '昨天新闻标题摘要'}
很抱歉，由于我无法直接访问互联网，无法为您提供昨天的新闻标题和摘要。不过，我可以给您提供一些建议，您可以尝试使用其他搜索引擎或新闻网站来查找相关信息。如果您需要进一步的帮助或有其他问题，请随时告诉我。


In [92]:
query = "还有哪些新闻？"
msgs.append(query,role="user")
chat(chat_comp, msgs, funcs, func_name_map)

**No function**
以下是昨天即2024年1月14日的一些新闻标题和摘要：

1. 中疾控：新冠疫情可能在本月出现回升，JN.1变异株大概率将成我国优势流行株。

摘要：中国疾病预防控制中心表示，根据目前的监测数据和模型预测，新冠疫情可能在本月出现回升。同时，JN.1变异株大概率将成为我国优势流行株，需要加强防控措施。

2. 31处雪崩，6公里路段被埋，最深超5米，游客滞留！喀纳斯景区：食宿免费。

摘要：新疆喀纳斯景区发生31处雪崩，导致6公里路段被埋，最深积雪超过5米。为确保游客安全，景区提供免费食宿和救援服务。

3. 时速400公里！成渝中线有望成跑得最快的高铁。

摘要：中国铁路总公司表示，成渝中线高铁设计时速达到400公里，有望成为全国跑得最快的高铁。该线路将连接成都和重庆两大城市，缩短两地间的旅行时间。

4. 美媒：红海局势升温正在推高全球航运价格，部分航线价格飙升。

摘要：美国媒体报道称，红海地区的紧张局势正在推高全球航运价格。一些航线价格飙升，对全球贸易造成负面影响。

5. 公司决定禁止融资买入证券用于偿还融券负债。

摘要：某公司宣布，自即日起禁止融资买入证券用于偿还融券负债。此举旨在规范公司融资行为，降低金融风险。

6. 挂钩海外上市的中国股票ETF—FXI期权近期出现新信号，看跌期权未平仓合约数量降至1年内低位。

摘要：据海外交易网站Market Chameleon信息显示，挂钩海外上市的中国股票ETF—FXI期权近期出现新信号。看跌期权未平仓合约数量降至1年内低位，这意味着一些空头逐步平仓离场。


In [93]:
query = "总结一下，这些新闻涉及了哪些主题？"
msgs.append(query,role="user")
chat(chat_comp, msgs, funcs, func_name_map)

**call function duckduckgo_search**
function arguments: {'search_query': '昨天新闻主题'}
根据您提供的新闻标题和摘要，这些新闻涉及了多个主题，包括新冠疫情、雪崩灾害、高铁建设、全球航运价格、公司融资行为以及海外上市的中国股票ETF等。这些新闻涵盖了社会、经济、交通、自然灾害等多个方面，反映了当前国内外的热点问题和关注焦点。
